# Extreme precipitation in the Canadian Arctic
Here, I process, clean and explore data used in the extreme precipitation in Canadian Arctic study.

Data are obtained from https://climatedata.ca/download/#station-download.  This is not the official ECCC site but collates and serves daily data.

_I need to check what they do_

Many stations have had site and instrumentation changes.  Where these have occurred and sites are co-located, records have been combined.

#### To Do
- Make plot of record span for each station, including periods of missing data  
- Make map of stations with symbols showing length of record
- Make table showing length of record and completeness of record by year  
- _Tair and Prceip statistics for combined records_  
- Plot heatmap showing timing and amount of precipitation for each station
- Calculate seasonal cycle of precipitations (precip days, average P, phase)  


In [17]:
from pathlib import Path
import re
import pprint
import json

import pandas as pd

DATAPATH = Path('/', 'home', 'apbarret', 'Data', 'Rain_on_snow', 'Canada_extreme_precip', 'Raw_station_files')

In [18]:
list(DATAPATH.glob('*.csv'))

[PosixPath('/home/apbarret/Data/Rain_on_snow/Canada_extreme_precip/Raw_station_files/2403497_climate_daily.csv'),
 PosixPath('/home/apbarret/Data/Rain_on_snow/Canada_extreme_precip/Raw_station_files/2400602_climate_daily.csv'),
 PosixPath('/home/apbarret/Data/Rain_on_snow/Canada_extreme_precip/Raw_station_files/2400800_climate_daily.csv'),
 PosixPath('/home/apbarret/Data/Rain_on_snow/Canada_extreme_precip/Raw_station_files/2202580_climate_daily.csv'),
 PosixPath('/home/apbarret/Data/Rain_on_snow/Canada_extreme_precip/Raw_station_files/2503649_climate_daily.csv'),
 PosixPath('/home/apbarret/Data/Rain_on_snow/Canada_extreme_precip/Raw_station_files/2403505_climate_daily.csv'),
 PosixPath('/home/apbarret/Data/Rain_on_snow/Canada_extreme_precip/Raw_station_files/2401203_climate_daily.csv'),
 PosixPath('/home/apbarret/Data/Rain_on_snow/Canada_extreme_precip/Raw_station_files/station_metadata.csv'),
 PosixPath('/home/apbarret/Data/Rain_on_snow/Canada_extreme_precip/Raw_station_files/2402354_